<a href="https://colab.research.google.com/github/yasaad/3630/blob/main/project4_student.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 4: Monte Carlo Localization

Due Date: Friday, March 26, 2021 @ 11:59 P.M.

Student Name: **Enter Name Here**

In this project, you will implement the particle filter and use it to improve the execution performance of a simulation robot tracking a path generated by RRT. Additionally, you will see how particle filters can help with the kidnapped robot problem. This is an individual assignment, and you will be held by the Georgia Tech honor code to finish it by yourself. Collaboration at the white board level is allowed with your teammates.

# Setup

**FIRST, visit Files on Canvas, download "project4_assets.zip" and upload it to the files tab (in the left bar). You will receive errors if you skip this step**

In this section, we'll install pip packages, import python modules, and download the required assets.

In [ ]:
# Delete this cell before exporting this notebook as a python file for submission
!pip install -q gdown plotly pybullet pathlib tqdm

!unzip -qo project4_assets.zip
!rm -rf duckie_msgs/ geometry_msgs/ nav_msgs/ std_msgs/ rclpy/
!mv project4_assets/mock_msgs/* ./
!mv project4_assets/urdf ./
!mv project4_assets/example_apartment.json ./
!mv project4_assets/helpers.py ./

In [ ]:
import copy
import math
import random
import numpy as np
from tqdm import tqdm

import rclpy
from rclpy.node import Node
from nav_msgs.srv import GetPlan
from nav_msgs.msg import Path
from geometry_msgs.msg import Point, Pose, PoseStamped
from duckie_msgs.msg import Wheels, Obstacle, ObstacleList, RangeBearingLandmark, RangeBearingLandmarkList

from helpers import *

## Helper Functions

You may use these functions within your code.

In [ ]:
def compute_relative(pose_i, pose_j):
    """Computes the transform T_i^j (from pose_i to pose_j)

    Parameters
    ----------
    pose_i: tuple(x, y, theta)
        a tuple containing (x, y, theta) values in world frame
    pose_j: tuple(x, y, theta)
        a tuple containing (x, y, theta) values in world frame

    Returns
    -------
    tuple(dx, dy, dtheta)
        the calculated transform T_i^j
    """
    x_i, y_i, theta_i = pose_i
    x_j, y_j, theta_j = pose_j
    delta_x_w = x_j - x_i
    delta_y_w = y_j - y_i
    delta_theta = theta_j - theta_i

    iRw = np.array([[np.cos(theta_i), np.sin(theta_i)], 
                        [-np.sin(theta_i), np.cos(theta_i)]])
    delta_xy_w = np.array([delta_x_w, delta_y_w])
    delta_xy_i = np.matmul(iRw, delta_xy_w)

    delta_x_i = delta_xy_i[0]
    delta_y_i = delta_xy_i[1]

    return (delta_x_i, delta_y_i, delta_theta)

In [ ]:
def pose_compose(T_i, T_j):
    """Computes the product of two transforms (T_i * T_j)
​
    Parameters
    ----------
    T_i: tuple(x, y, theta)
        a tuple containing (x, y, theta) values for the transform
    T_j: tuple(x, y, theta)
        a tuple containing (x, y, theta) values for the transform
​
    Returns
    -------
    tuple(dx, dy, dtheta)
        the calculated transform T_i*T_j
    """
    x_i, y_i, theta_i = T_i
    x_j, y_j, theta_j = T_j
    result_theta = theta_j + theta_i
    R_i = np.array([[np.cos(theta_i), -np.sin(theta_i)], 
                    [np.sin(theta_i), np.cos(theta_i)]])
    xy_j = np.array([x_j, y_j])
    xy_result = np.matmul(R_i, xy_j)
    result_x = x_i + xy_result[0]
    result_y = y_i + xy_result[1]
    return (result_x, result_y, result_theta)

# Visualization

In this section, we'll take a look at the simulator/visualizer. The `Simulator` class manages the physics simulation and visualization of the differential drive robot. It also provides methods for interacting with the environment that you will use in your implementation.

Below, the cell creates a `sim` object that reads information about the map from the "example_apartment.json" file. You may look at the Files tab (in the left bar) to upload and use your own map files. The visualized map shows obstacles in black, landmarks in yellow, and open space in white. You may pan and zoom using the menu bar.

In [ ]:
sim = Simulator('example_apartment.json')
sim.visualize()

We can command the robot via the `sim.step()` function. It takes in a duckie_msgs/Wheels.msg object which gives the robot left and right wheel commands.

Below, we command the robot to move forward for 10 seconds. You may press the play button to see the simulation. You may also scrub through the timeline manually. Note that pressing the play button multiple times will not pause the playback.

You can adjust the video playback speed with the `duration` parameter on the `sim.visualize()` method. Because it is set to 0.1 seconds here, each simulated timestep (1 second here) is visualized in 0.1 seconds in the video.

In [ ]:
sim = Simulator('example_apartment.json', load_landmarks=False)

# simulate out 10 seconds total (10 iteration, 1sec per iteration)
for _ in range(10):
    sim.step(Wheels(left_wheel=3.0, right_wheel=3.0), None, duration=1.0)

sim.visualize(duration=0.1) # visualize all 10 simulated seconds in 1 second

You may experiment with

- the robot's initial pose in the simulation (`initial_pose` parameter)
- the noise on the odometry/measurements (`odom_sigma_*` and `marker_sigma_*` parameters)
- whether the landmarks are loaded in (`load_landmarks` parameter)
- whether the grid lines are shown (`show_grid_lines` parameter)

Below, a cell demonstrates how to change these parameters.

In [ ]:
sim = Simulator('example_apartment.json', load_landmarks=True,
                odom_sigma_x=1.0, odom_sigma_y=1.0, odom_sigma_h=1.0, 
                marker_sigma_range=1.0, marker_sigma_bearing=1.0,
                initial_pose=[2.0, 3.0, -np.pi/2])

with tqdm(total=40) as pbar:
    for _ in range(40):
        sim.step(Wheels(left_wheel=3.0, right_wheel=3.0), None, duration=1.0)
        pbar.update()

sim.visualize(show_grid_lines=True, duration=0.1)

Additionally, the simulator has utility functions that you will need for your implementation for this assignment.

- `sim.get_obstacles()` - returns a list of obstacles in the map
- `sim.is_free(x, y)` - boolean whether a circle centered at (x, y) with radius 0.15m intersects with any obstacles/walls
- `sim.visible_markers_gt(x, y, theta)` - given a pose, this method will return a list of landmarks visible to a robot at this position.

In [ ]:
print("Obstacles: ", [(ob.x, ob.y, ob.width, ob.height) for ob in sim.get_obstacles().obs])
print("Free at (-1, -1) Expected False: ", sim.is_free(-1, -1))
print("Free at (1, 2) Expected False: ", sim.is_free(1, 2))
print("Free at (1, 1) Expected True: ", sim.is_free(1, 1))
print("Markers Visible at (3, 1, PI/2): ", [rb.id for rb in sim.visible_markers_gt([3, 1, np.pi / 2])])

# Planning

You will not need to implement any code in this section. This section recaps the findings from the RRT project. In the cell below, a hardcoded version of the `RRTPlanner` service provided a path from (1, 1) to (6.2, 3.0) with 5 waypoints total. You may directly copy and paste your solution for RRT project into this cell to have your robot be able to plan for any valid start and goal points in the map. You may copy the contents of the `rrt_planner.py` class verbatim into the cell below.

To be clear, we are not running ROS2 code in Google Colab, but rather we have mocked out the ROS2 rclpy library and message modules. 

In [ ]:
# You may replace this hardcoded RRTPlanner with your RRT implementation from Project 3

class RRTPlanner():

    def plan_callback(self, request, response):
        response.plan.poses = [
            PoseStamped(pose=Pose(position=Point(x=1.0, y=1.0))),
            PoseStamped(pose=Pose(position=Point(x=4.0, y=0.8))),
            PoseStamped(pose=Pose(position=Point(x=5.0, y=1.2))),
            PoseStamped(pose=Pose(position=Point(x=5.2, y=3.2))),
            PoseStamped(pose=Pose(position=Point(x=6.2, y=3.0))),
        ]
        return response

We form a requst using the `GetPlan.Request()` initializer. We give it start and goal points in the map, and a tolerance within which the RRT algorithm can find a goal state.

In the cell below, we provide the RRTPlanner `rrt_planner` with obstacles from `sim.get_obstacles()`. Then we trigger the planning callback and visualize the plan using `sim.visualize(plan=result)`.

In [ ]:
# form request
request = GetPlan.Request()
start_point = Point(x=1.0, y=1.0)
goal_point = Point(x=6.2, y=3.0)
request.start = PoseStamped(pose=Pose(position=start_point))
request.goal = PoseStamped(pose=Pose(position=goal_point))
request.tolerance = 0.1 # meters

# load obstacles and trigger planning
response = GetPlan.Response()
sim = Simulator('example_apartment.json', load_landmarks=False)
rrt_planner = RRTPlanner()
rrt_planner.obstacles = sim.get_obstacles()
response = rrt_planner.plan_callback(request, response)

# visualize plan
sim.visualize(plan=response.plan, show_grid_lines=False)

In the following cell, we provide an entirely implemented NaiveController. The controller has a function called `execute_plan()` that is called once per iteration. It is provided a relative odometry update (provided by `sim.step()`). The approach to path tracking implemented in NaiveController is often called Dead Reckoning.

In [ ]:
class NaiveController:

    def __init__(self, plan, initial_pose=[0.0, 0.0, 0.0], max_rotational_speed=1.0, max_translation_speed=2.5):
        self.plan = plan
        self.goal_waypoint_idx = 0
        self.rotation_speed = max_rotational_speed
        self.translation_speed = max_translation_speed
        self.pose = initial_pose

    def _within_tolerance_of_point(self, x, y, waypoint, tolerance_m):
        def distance(x1, y1, x2, y2):
            return ((x1 - x2) ** 2 + (y1 - y2) ** 2) ** 0.5

        return distance(waypoint.x, waypoint.y, x, y) <= tolerance_m

    def execute_plan(self, rel_pose):
        # integrate rel_pose
        self.pose = pose_compose(self.pose, rel_pose)

        # check if arrived to waypoint
        if self.goal_waypoint_idx < len(self.plan.poses) and \
           self._within_tolerance_of_point(self.pose[0], self.pose[1], self.plan.poses[self.goal_waypoint_idx].pose.position, tolerance_m=0.05):
            self.goal_waypoint_idx += 1

        # check if execution complete
        if self.goal_waypoint_idx >= len(self.plan.poses):
            return Wheels() # stop command

        # perform rotation if required
        robot_frame = np.array([[np.cos(self.pose[2]), - np.sin(self.pose[2]), self.pose[0]],
                                [np.sin(self.pose[2]),   np.cos(self.pose[2]), self.pose[1]],
                                [0.0,                    0.0,                  1.0]])
        goal_waypoint = self.plan.poses[self.goal_waypoint_idx].pose.position
        goal_waypoint_arr = np.array([goal_waypoint.x, goal_waypoint.y, 1.0]).T
        projected = np.dot(np.linalg.inv(robot_frame), goal_waypoint_arr) # use .T TODO
        goal_heading = np.arctan2(projected[1], projected[0])
        # print(projected, goal_heading)
        if not np.isclose(goal_heading, 0.0, atol=0.15):
            return Wheels(left_wheel=-np.sign(goal_heading) * self.rotation_speed * (abs(goal_heading) / 1.57),
                          right_wheel=np.sign(goal_heading) * self.rotation_speed * (abs(goal_heading) / 1.57))

        # perform translation
        return Wheels(left_wheel=self.translation_speed, right_wheel=self.translation_speed)


We show how this NaiveController is able to get the robot to the goal when given ideal conditions (no noise and perfect world knowledge).

In [ ]:
# Execute planned path
sim = Simulator('example_apartment.json', load_landmarks=False, odom_sigma_x=0.0, odom_sigma_y=0.0, odom_sigma_h=0.0)
controller = NaiveController(response.plan, initial_pose=[1.0, 1.0, 0.0])
cmd = Wheels(left_wheel=0.0, right_wheel=0.0)
with tqdm(total=125) as pbar:
    for _ in range(125): # simulate out 125 seconds
        odom, _, _ = sim.step(cmd, None, duration=1.0)
        cmd = controller.execute_plan(odom)
        pbar.update()

sim.visualize(plan=response.plan, show_grid_lines=True)

But what happens when we stop assuming no noise?

In [ ]:
ODOM_NOISE_SIGMA_X = 4e-3
ODOM_NOISE_SIGMA_Y = 4e-3
ODOM_NOISE_SIGMA_HEADING = 8e-3

# Execute planned path
sim = Simulator('example_apartment.json', load_landmarks=False,
                odom_sigma_x=ODOM_NOISE_SIGMA_X, odom_sigma_y=ODOM_NOISE_SIGMA_Y,
                odom_sigma_h=ODOM_NOISE_SIGMA_HEADING)
controller = NaiveController(response.plan, initial_pose=[1.0, 1.0, 0.0])
cmd = Wheels(left_wheel=0.0, right_wheel=0.0)
with tqdm(total=125) as pbar:
    for _ in range(125): # simulate out 125 seconds
        odom, _, _ = sim.step(cmd, None, duration=1.0)
        cmd = controller.execute_plan(odom)
        pbar.update()

sim.visualize(plan=response.plan, show_grid_lines=True)

Also note that we explicitly had to tell the NaiveController that the robot started at the initial position in world frame of (1.0, 1.0). We'll see how the particle filter will enable us to automatically figure out our position.

# Particle Filter

In this section, we'll implement the particle filter. The cell below provides the template for it. Below it are test and scenarios to help test your particle filter.

In [ ]:
class ParticleFilter:

    def __init__(self, sim, num_particles=5000, injected_xy_sigma=0.1, injected_heading_sigma=0.1, range_sigma=0.5, bearing_sigma=0.3):
        """
        Constructor

        Parameters
        ----------
        sim: Simulator
            simulation environment
        num_particles: int
            number of particles to keep in the particle filter
        injected_xy_sigma: float
            sigma of additional noise to inject in (x, y) components during motion update
        injected_heading_sigma: float
            sigma of additional noise to inject in the heading component during motion update
        range_sigma: float
            sigma of noise in landmark range measurement
        bearing_sigma: float
            sigma of noise in landmark bearing measurement
        """
        self.sim = sim
        self.num_particles = num_particles
        self.range_sigma = range_sigma
        self.bearing_sigma = bearing_sigma
        self.injected_xy_sigma = injected_xy_sigma
        self.injected_heading_sigma = injected_heading_sigma
        self.prev_x = 0.0
        self.prev_y = 0.0
        self.prev_heading = 0.0
        self.particles = []
        self.initialize_particles()

    def generate_random_particles(self, num_particles):
        """
        Generate particles in random locations within the free space of map

        Parameters
        ----------
        num_particles: int
            number of particles to generate
        
        Return
        ----------
        Numpy array of shape (N, 3), where N is the number of particles and each
            particles has an x (meters), y (meters), and heading (radians) component.
            Particles are given with respect to the map frame (i.e. the bottom left
            corner of the map)
        """
        particles = []
        while len(particles) < num_particles:
            particle_x = np.random.uniform(0.15, self.sim.env_manager.map_json['room']['xsize_m'] - 0.15)
            particle_y = np.random.uniform(0.15, self.sim.env_manager.map_json['room']['ysize_m'] - 0.15)
            if (self.sim.is_free(particle_x, particle_y)):
                particle_h = np.random.uniform(-np.pi, np.pi)
                particle = (particle_x, particle_y, particle_h)
                particles.append(particle)
        return np.array(particles)

    def initialize_particles(self):
        """
        Initialize the particles in random locations within the free space of the map
        """
        self.particles = self.generate_random_particles(self.num_particles)

    def compute_estimate(self):
        """
        Compute the pose estiamte as the average of all particles
        """
        particles = np.array(self.particles)
        x_arr = particles[:, 0]
        y_arr = particles[:, 1]
        h_arr = particles[:, 2] 
        avg_x = np.average(x_arr)
        avg_y = np.average(y_arr)
        avg_cos = np.average(np.cos(h_arr))
        avg_sin = np.average(np.sin(h_arr))
        avg_h = np.arctan2(avg_sin, avg_cos)
        return (avg_x, avg_y, avg_h)

    def is_confident(self):
        """
        provide the confidence of esitmation by checking if 95% of particles are close
        to the average pose
        """
        avg_pose = self.compute_estimate()
        count = 0
        for particle in self.particles:
            diff_x = abs(particle[0] - avg_pose[0])
            diff_y = abs(particle[1] - avg_pose[1])
            diff_theta = abs(particle[2] - avg_pose[2])
            if (diff_theta > np.pi):
                diff_theta = 2 * np.pi - diff_theta
            if (diff_x ** 2 + diff_y ** 2) ** 0.5 < 1 and diff_theta < 0.5:
                count += 1
        return count / self.num_particles >= 0.95

    def motion_update(self, rel_pose, step_sigma):
        """Move all particles by the given odometry.

        Parameters
        ----------
        rel_pose: tuple of (float, float, float)
            (x, y, theta) represeting the relative pose between the current step (k) and the 
            previous step (k-1) (T_{k-1}^k)
        step_sigma: tuple of (float, float, float)
            (sigma_x, sigma_y, sigma_theta) represeting the standard deviation for the noise 
            in the x, y, theta components of the relative pose. Assume the noise in x, y, theta 
            are independent

        Effect
        ----------
            Update self.particles so that each particle has moved by the given odometry and some 
            noise.

        Notes
        -----
        - Be sure to transform the odometry to each particle's frame. You may find the helper 
            function "pose_compose" helpful
        - Don't worry about particles leaving the grid/entering obstacles here. You
            will give these particles weights of zero to ensure they aren't sampled.
        - self.particles is a Numpy array of shape (N, 3), where N is the number of particles and 
            each particles has an x (meters), y (meters), and heading (radians) component.
        - Use np.random.normal to add noise.
        - When calculating sigma to apply noise to the particle's motion update,
            include `self.xy_sigma` and `self.heading_sigma` as well as the sigmas
            passed in from the simulator.
        """
        # STUDENT TODO
        return

    def measurement_update(self, measured_marker_list):
        """ Weight the particles by measurements, and resample particles according to their weights

        Parameters
        ----------
        measured_marker_list : list of RangeBearingLandmark object
            all bearing range measurements (with noise) of markers collected by the robot at this step
            each RangeBearingLandmark object has 3 attributes:
                RangeBearingLandmark.id : string (type of the marker)
                RangeBearingLandmark.range : double (range measurement)
                RangeBearingLandmark.bearing : double (bearing measurement)
        
        Effect
        ----------
            Weight the particles by measurements, and resample particles according to their weights. Create 
            a small fraction of random particles and add to self.particles.

        Notes
        -------
        - You can call the "self.sim.visible_markers_gt(pose)" to get the bearing range measurements (without noise) 
            of all visible landmarks at the specified pose (pose is specified by the tuple (x, y, theta)). The 
            function returns the same type as "measured_marker_list".
        - Markers are associated with an 'id' that identifies the type of the marker. Multiple
            markers can have the same id. If multiple markers with the same id are observed, you need to perform 
            matching across landmarks
        - If there are markers cannot be matched, you can simply set the weight of the corresponding particle to 0
        - You can add a small fraction of additional randomly sampled particles with "generate_random_particles" 
            function

        """
        # Student TODO
        return



We have provided a function called `initialize_particles()` to initialize the particles randomly at valid locations in the map.

In this cell, you'll visualize what this initialization of particles looks like. You may play with the number of particles by passing in a number to the `num_particles` parameter in the ParticleFilter class constructor. Note that each particle has an orienatation in the map, however, we do not visualize the angle of the particles here. 

In [ ]:
sim = Simulator('example_apartment.json')

pf = ParticleFilter(sim, num_particles=200)
pf.initialize_particles()

sim.step(Wheels(), pf.particles)
sim.visualize(show_grid_lines=True)

###Task 2: Motion Update

Implement the `motion_update()` method in the ParticleFilter class. Notice that you can compute the pose of current step $T_w^k$ ($w$ represents world frame) from the pose of the previous step $T_w^{k-1}$ and the odometry(relative pose) $T_{k-1}^k$ by

$T_w^k=T_w^{k-1} T_{k-1}^k$

Below, we provide you a code snippet that enables you to run motion updates with hardcoded motions. Because we have turned off noise, you should expect to see the particles only move forward in a straight line.

You may modify the hardcoded commands to see other motions. You may add noise to the simulator (which determine that values for `sigmas` returned from `sim.step()`) to see how the particles move with noise.

In [ ]:
sim = Simulator('example_apartment.json', odom_sigma_x=0.0, odom_sigma_y=0.0, odom_sigma_h=0.0)

pf = ParticleFilter(sim, num_particles=10, injected_xy_sigma=0.0, injected_heading_sigma=0.0)
pf.initialize_particles()

odom, sigmas, _ = sim.step(Wheels(left_wheel=1.0, right_wheel=1.0), pf.particles, duration=2.0)
pf.motion_update(odom, sigmas)
odom, sigmas, _ = sim.step(Wheels(left_wheel=1.0, right_wheel=1.0), pf.particles, duration=2.0)
pf.motion_update(odom, sigmas)
odom, sigmas, _ = sim.step(Wheels(left_wheel=1.0, right_wheel=1.0), pf.particles, duration=2.0)
pf.motion_update(odom, sigmas)
odom, sigmas, _ = sim.step(Wheels(left_wheel=1.0, right_wheel=1.0), pf.particles, duration=2.0)
pf.motion_update(odom, sigmas)
odom, sigmas, _ = sim.step(Wheels(left_wheel=0.0, right_wheel=0.0), pf.particles, duration=1.0)

sim.visualize(show_grid_lines=True, duration=0.5)

###Task 3: Measurement Update

Implement the `measurement_update()` method in the ParticleFilter class. Note that to the probablity density for a bearing range measurement $(b, r)$ with ground truth $(b_{gt}, r_{gt})$ is provided by

$P(b,r)\propto exp({\frac{(b-b_{gt})^2}{\sigma_b^2/2}}) * exp({\frac{(r-r_{gt})^2}{\sigma_r^2/2}})$

Below, we provide you a code snippet that enables you to run measurement updates with hardcoded motions. You should be able to see the particles converge on the ground truth position of the robot as the measurements are used to filter out unlikely particles.

You may modify the hardcoded commands to see other motions. You may play with the injected noise (`xy_sigma` and `heading_sigma` parameters below) in the motion update. You should not change the odometry and measurement noise given in each example below (we would not be able to change it with a real robot because it would be determined by hardware/sensors).

In [ ]:
ODOM_NOISE_SIGMA_X = 4e-4
ODOM_NOISE_SIGMA_Y = 4e-4
ODOM_NOISE_SIGMA_HEADING = 8e-4
MEASURE_NOISE_SIGMA_RANGE = 0.5
MEASURE_NOISE_SIGMA_BEARING = 0.3

sim = Simulator('example_apartment.json', odom_sigma_x=ODOM_NOISE_SIGMA_X, odom_sigma_y=ODOM_NOISE_SIGMA_Y,
                odom_sigma_h=ODOM_NOISE_SIGMA_HEADING, marker_sigma_range=MEASURE_NOISE_SIGMA_RANGE,
                marker_sigma_bearing=MEASURE_NOISE_SIGMA_BEARING, initial_pose=[5.5, 3.3, 0.0])

pf = ParticleFilter(sim, num_particles=500, injected_xy_sigma=0.1, injected_heading_sigma=0.1,
                    range_sigma=MEASURE_NOISE_SIGMA_RANGE, bearing_sigma=MEASURE_NOISE_SIGMA_BEARING)
pf.initialize_particles()

with tqdm(total=40) as pbar:
    for _ in range(40): # simulate out 20 seconds
        odom, sigmas, measured_markers = sim.step(Wheels(left_wheel=2.5, right_wheel=1.0), pf.particles, duration=0.5)
        pf.motion_update(odom, sigmas)
        pf.measurement_update(measured_markers)
        pbar.update()

sim.visualize(show_grid_lines=True, duration=0.25)

The scenario here is similar to the circular motion shown in the cell above. Here we see the robot moving left from the right handside of the room.

In [ ]:
ODOM_NOISE_SIGMA_X = 4e-4
ODOM_NOISE_SIGMA_Y = 4e-4
ODOM_NOISE_SIGMA_HEADING = 8e-4
MEASURE_NOISE_SIGMA_RANGE = 0.5
MEASURE_NOISE_SIGMA_BEARING = 0.3

sim = Simulator('example_apartment.json', odom_sigma_x=ODOM_NOISE_SIGMA_X, odom_sigma_y=ODOM_NOISE_SIGMA_Y,
                odom_sigma_h=ODOM_NOISE_SIGMA_HEADING, marker_sigma_range=MEASURE_NOISE_SIGMA_RANGE,
                marker_sigma_bearing=MEASURE_NOISE_SIGMA_BEARING, initial_pose=[8.0, 1.3, np.pi])

pf = ParticleFilter(sim, num_particles=500, injected_xy_sigma=0.1, injected_heading_sigma=0.1,
                    range_sigma=MEASURE_NOISE_SIGMA_RANGE, bearing_sigma=MEASURE_NOISE_SIGMA_BEARING)
pf.initialize_particles()

with tqdm(total=50) as pbar:
    for _ in range(50): # simulate out 50 seconds
        odom, sigmas, measured_markers = sim.step(Wheels(left_wheel=2.0, right_wheel=2.0), pf.particles, duration=1.0)
        pf.motion_update(odom, sigmas)
        pf.measurement_update(measured_markers)
        pbar.update()

sim.visualize(show_grid_lines=True, duration=0.5)

# Execution

### Task 4: Path Tracking with an Informed Controller

Implement the `execute_plan` method in the InformedController class below. You may refer to the NaiveController class implemented provided in the "Planning" section. Note that NaiveController's `execute_plan()` method is given relative odometry updates (dx, dy, dtheta), whereas the InformedController's is given a pose estimate in the world frame (x_w, y_w, theta_w) that is estimated by the particle filter. Additionally, it is given a boolean `is_confident`, which tells you whether at least 95% of the particles agreed with the mean pose of all particles. You should use the `is_confident` boolean to change the behavior of your robot. If you are not confident about where you are, use a hard-coded convergence method (see two examples in Task 3 above) until the particle filter has seen enough measurements to give more confident estimates. Once confident, start execution of the path. If you lose confidence while tracking the path, revert to the search mode once more.

In [ ]:
class InformedController:

    def __init__(self, plan, max_rotational_speed=1.0, max_translation_speed=2.5):
        self.plan = plan
        self.rotation_speed = max_rotational_speed
        self.translation_speed = max_translation_speed

    def _within_tolerance_of_point(self, x, y, waypoint, tolerance_m):
        def distance(x1, y1, x2, y2):
            return ((x1 - x2) ** 2 + (y1 - y2) ** 2) ** 0.5

        return distance(waypoint.x, waypoint.y, x, y) <= tolerance_m

    def execute_plan(self, pose, is_confident):
        """
        compute the wheel commands for a small time step to follwo the planned trajectory

        Parameters
        --------
        pose : (float, float, float)
            estimate of current pose in (x, y, theta)
        is_confident : bool
            if the current pose estiamte is confident
        """
        # STUDENT TODO
        return Wheels(left_wheel=0.0, right_wheel=0.0)
        


Below we provide a complete pipeline that robot uses to plan the path with RRT, estimate its pose with Monte Carlo Localization, and path tracking. You should see your robot able to execute the path despite
1. being given **no** information on where the robot started (the kidnapped robot problem)
2. significant noise in the odometry and measurements.

In [ ]:
# create the simulator for the "example_apartment" map
sim = Simulator('example_apartment.json')

# create RRTPlanner service request/response objects
request = GetPlan.Request()
start_point = Point(x=1.0, y=1.0)
goal_point = Point(x=6.2, y=3.0)
request.start = PoseStamped(pose=Pose(position=start_point))
request.goal = PoseStamped(pose=Pose(position=goal_point))
request.tolerance = 0.1 # meters
response = GetPlan.Response()

# load obstacles and trigger planning
duckie_rrt_planner = RRTPlanner()
duckie_rrt_planner.obstacles = sim.get_obstacles()
response = duckie_rrt_planner.plan_callback(request, response)

# initialize particle filter
pf = ParticleFilter(sim, num_particles=1000)
pf.initialize_particles()

# execute planned path
controller = InformedController(response.plan)
cmd = Wheels(left_wheel=0.0, right_wheel=0.0)
with tqdm(total=100) as pbar:
    for t in range(100): # simulate out 50 seconds
        odom, sigmas, measured_markers = sim.step(cmd, pf.particles, duration=0.5)
        pf.motion_update(odom, sigmas)
        pf.measurement_update(measured_markers)
        cmd = controller.execute_plan(pf.compute_estimate(), pf.is_confident())
        pbar.update()

# visualize the robot, plan, and particles
sim.visualize(plan=response.plan, show_grid_lines=True)

**Take a video of the animation generated by the cell above. Submit this video as a project deliverable.**

# Reflection & Submission

Complete the reflection questions in the proj4_report_template.pptx, which you can find in the files tab on Canvas. Failure to follow the format will be penalized. Save the report as a .pdf and rename it to "FIRSTNAME_LASTNAME_reflection.pdf".

### Rubric

 * 30 points: Particle Filter Implementation
   - 10 points: Motion Update
   - 20 points: Measurement Update
 * 20 points: Informed Controller
 * 30 points: Demo Video
 * 20 points: Reflection

### Submission Details

Deliverables are a zip file named "FIRSTNAME_LASTNAME_project4.zip" with the following files:

 * project4.py - exported from Google Colab (File -> Download .py)
 * FIRSTNAME_LASTNAME_reflection.pdf - exported from the given powerpoint
 * FIRSTNAME_LASTNAME_video.{mov, mp4} - No longer than 30 seconds

This is an individual assignment, everyone should submit their own files.

